In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta, timezone
import requests
import os
from time import sleep
import pytz
from scipy.stats import pearsonr
from Functions import check_correlation, analyze_events, wrap_analyze_events, load_data, data_preparation, wrap_analyze_events_multiprocessing, show_output, start_wrap_analyze_events_multiprocessing, get_volume_info
import pandas as pd
pd.set_option('display.max_rows', None)
from time import time
from multiprocessing import Process
from multiprocessing import Lock, Pool, Manager
from time import time

/home/alberto/Docker/Trading/analysis/Functions.py:188: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if key is not 'coins' or key is not 'events':
/home/alberto/Docker/Trading/analysis/Functions.py:188: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if key is not 'coins' or key is not 'events':


In [2]:
start_interval=datetime(2023,5,11)
end_interval=datetime.now()

filter_position=(0,100)

data, df, df_obj = load_data(start_interval=start_interval, end_interval=end_interval, filter_position=filter_position)
df

/home/alberto/Docker/Trading/analysis/benchmark_json/benchmark-24-5-2023 exists. Loading the file...
['/home/alberto/Docker/Trading/analysis/json/data-22-5-2023-9206.json']
Retrieving data from /home/alberto/Docker/Trading/analysis/json/data-22-5-2023-9206.json
100 coins have been loaded


,n_observations,position,vol_30_avg,std_on_mean,first_obs
BTCUSDT,20035,0,933959.46,1.527535,2023-05-11T00:00:05.857211
ETHUSDT,20035,1,526068.65,1.789271,2023-05-11T00:00:05.857211
PEPEUSDT,20029,2,95980.15,1.592035,2023-05-11T00:00:05.857211
XRPUSDT,20029,3,92913.56,1.38438,2023-05-11T00:00:05.857211
ARBUSDT,20029,4,80313.53,1.533886,2023-05-11T00:00:05.857211
BNBUSDT,20035,5,76356.34,1.809824,2023-05-11T00:00:05.857211
SUIUSDT,20034,6,57264.19,1.600413,2023-05-11T00:00:05.857211
EDUUSDT,20028,7,53866.96,1.617779,2023-05-11T00:00:05.857211
SOLUSDT,20035,9,46212.43,2.305967,2023-05-11T00:00:05.857211
LTCUSDT,20035,10,41283.1,1.930737,2023-05-11T00:00:05.857211


In [ ]:
# WRAP ANALYSIS EVENTS
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m', 'buy_vol_24h']
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m', 'vol_24h']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m']
list_minutes = [5, 15, 30, 60, 120, 180, 360]
list_event_buy_volume = [0.7, 0.8]
list_event_volume = [5,6]

resp = wrap_analyze_events(data, list_buy_vol, list_vol, list_minutes, list_event_buy_volume,
                    list_event_volume, start_interval, end_interval)


In [3]:
# WRAP ANALYSIS EVENTS MULTIPROCESSING
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m', 'buy_vol_24h']
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m', 'vol_24h']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m']
list_minutes = [5, 15, 30, 60, 120, 180, 360]
list_event_buy_volume = [0.65, 0.7, 0.8]
list_event_volume = [4,5,6]
n_processes = 6

shared_data = start_wrap_analyze_events_multiprocessing(data, list_buy_vol, list_vol, 
                                                        list_minutes, list_event_buy_volume,
                                                        list_event_volume, start_interval,
                                                        end_interval, n_processes)


35 coins will be analyzed from 2023-05-11T00:00:05.857211 to 2023-05-25T06:06:02.654766
total_combinantions :  1008


/home/alberto/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/alberto/.local/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/alberto/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/alberto/.local/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/alberto/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/alberto/.local/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret

86.7093677520752  seconds


In [4]:
# SHOW DATA FROM MULTIPROCESSING
output = show_output(shared_data)
df = pd.DataFrame(output).transpose()
df.sort_values(by=['lower_bound'], ascending=False)

,mean,std,lower_bound,n_coins,n_events
buy_vol_15m:0.8/vol_60m:4/timeframe:60,9.578848,0.000000,9.578848,1.0,1.0
buy_vol_15m:0.8/vol_30m:6/timeframe:60,9.510920,0.000000,9.510920,1.0,1.0
buy_vol_15m:0.8/vol_60m:4/timeframe:120,9.480304,0.000000,9.480304,1.0,1.0
buy_vol_15m:0.8/vol_30m:6/timeframe:120,9.458398,0.000000,9.458398,1.0,1.0
buy_vol_15m:0.8/vol_30m:6/timeframe:180,9.399723,0.000000,9.399723,1.0,1.0
buy_vol_15m:0.8/vol_60m:4/timeframe:180,9.390991,0.000000,9.390991,1.0,1.0
buy_vol_15m:0.8/vol_30m:6/timeframe:5,9.244540,0.000000,9.244540,1.0,2.0
buy_vol_15m:0.8/vol_30m:6/timeframe:360,9.242644,0.000000,9.242644,1.0,1.0
buy_vol_15m:0.8/vol_60m:4/timeframe:360,9.242091,0.000000,9.242091,1.0,1.0
buy_vol_15m:0.8/vol_60m:4/timeframe:5,9.166147,0.000000,9.166147,1.0,1.0


In [ ]:
df = pd.DataFrame(resp).transpose()
lb = []
for index, row in df.iterrows():
    mean = row['mean']
    std = row['std']
    lb.append(mean - std)
    
df['lb'] = lb
    
df.sort_values(by=['lb'], ascending=False)

In [ ]:
output = show_output(shared_data)
df = pd.DataFrame(output).transpose()
df